***GENERATED CODE FOR 171023manualfet PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import pandas as pd
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        hdfs = HDFileSystem(host=hdfs_server, port=hdfs_port)
        #hdfs = HDFileSystem(host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            dfPd = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run 171023manualfetHooks.ipynb
try:
	#sourcePreExecutionHook()

	telcocutomerchurntetl = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1697004882843_Telco_Customer_Churn_TestL.csv', 'filename': 'Telco_Customer_Churn_TestL.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/ManualPipelines/Classification_Manual/Telco_Customer_Churn_TestL.csv', 'viewFileName': 'Telco_Customer_Churn_TestL.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run 171023manualfetHooks.ipynb
try:
	#transformationPreExecutionHook()

	manualfe = TransformationMain.run(telcocutomerchurntetl,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "customerID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "30", "mean": "", "stddev": "", "min": "0280-XJGEX", "max": "9959-WOFKT", "missing": "0", "distinct": "30"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "customerID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "30", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SeniorCitizen", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "30", "mean": "0.03", "stddev": "0.18", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "SeniorCitizen"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Partner", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "30", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Partner"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Dependents", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "30", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Dependents"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "tenure", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "30", "mean": "29.53", "stddev": "23.97", "min": "1", "max": "72", "missing": "0"}, "updatedLabel": "tenure"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PhoneService", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "30", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PhoneService"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MultipleLines", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "30", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MultipleLines"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InternetService", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "30", "mean": "", "stddev": "", "min": "DSL", "max": "No", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "InternetService"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "OnlineSecurity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "30", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "OnlineSecurity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "OnlineBackup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "30", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "OnlineBackup"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DeviceProtection", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "30", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DeviceProtection"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TechSupport", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "30", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TechSupport"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "StreamingTV", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "30", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "StreamingTV"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "StreamingMovies", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "30", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "StreamingMovies"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Contract", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "30", "mean": "", "stddev": "", "min": "Month-to-month", "max": "Two year", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Contract"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PaperlessBilling", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "30", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PaperlessBilling"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PaymentMethod", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "30", "mean": "", "stddev": "", "min": "Bank transfer (automatic)", "max": "Mailed check", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PaymentMethod"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MonthlyCharges", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "30", "mean": "64.54", "stddev": "31.08", "min": "18.95", "max": "113.25", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MonthlyCharges"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TotalCharges", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "30", "mean": "2238.17", "stddev": "2330.22", "min": "20.15", "max": "7895.15", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TotalCharges"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Churn", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "30", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Churn"}]}))

	#transformationPostExecutionHook(manualfe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run 171023manualfetHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(manualfe, ["customerID", "gender", "SeniorCitizen", "Partner", "Dependents", "tenure", "PhoneService", "MultipleLines", "InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod", "MonthlyCharges", "TotalCharges"], "Churn")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)
spark.stop()


